# Handmatig overhalen telefonische contacten Intake en planning tbv KCM (inmiddels geautomatiseerd)

WORKAROUND TOTDAT DATA AUTOMATISCH WORDT VERWERKT IN TS\_DATA

- stap 1: haal data op van postgres
- stap 2: verwerk dit in sql server in een tussentabel
- stap 3: dagelijks loopt rapport-schedule die data klaarzet

Jaco van der Wel, 5 juli 2021

## Data van postgres (iris) =\> <mark>connectie A-BI-PROD</mark>

In [1]:
SELECT cas.kubion_id AS case_id
    , cas.code
    , cas.description
    , cas.created_datetime
    , cas.lastchanged_datetime
    , cas.direction_id
    , dir.description AS direction
    , cas.channel_id
    , chnl.description AS channel
    , cas.subject_id
    , subj.code AS subject_code
    , subj.description AS subject
    , cas.status_id
    , stat.label AS casestatus
    , aut.externid AS created_user_wrkn
    , aut.fullname AS created_user_fullname
    , idobj.kubion_id AS index_object_id
    , idobj.contract_id AS huurder_oge
    , idobj.relation_id AS huurder_rlts
    ,aut.department
FROM vault.facts.fac_iris4_case cas
INNER JOIN vault.dimensions.dim_iris4_direction dir
    ON cas.direction_id = dir.kubion_id
        AND dir.description = 'Huurder'
INNER JOIN vault.dimensions.dim_iris4_channel chnl
    ON cas.channel_id = chnl.kubion_id
        AND chnl.description = 'Telefoon'
INNER JOIN vault.dimensions.dim_iris4_subject subj
    ON cas.subject_id = subj.kubion_id
        AND subj.description = 'DAGEONDERH Reparatieverzoek'
INNER JOIN vault.dimensions.dim_iris4_casestatus stat
    ON cas.status_id = stat.kubion_id
        AND stat.label = 'Afgehandeld'
INNER JOIN vault.dimensions.dim_iris4_autuser aut
    ON cas.created_user_id = aut.kubion_id
INNER JOIN vault.facts.fac_iris4_indexobject idobj
    ON cas.kubion_id = idobj.case_id
INNER JOIN vault.dimensions.dim_empire_customers cust
    ON cust.contact_no = idobj.relation_id
        AND cust.DATE_TO = '2199-12-31 23:59:59.999'
     --   where aut.fullname  = 'Ingrid Moerman'
WHERE cas.created_datetime >= '2021-07-01 00:00:00.999' 
    AND cas.description IN ('Reparatieverzoek met planning geregistreerd', 'Reperatieverzoek geregistreerd zonder planning')
    AND aut.department = 'Intake'
    --'2019-09-23'
    --DATE_TRUNC('day', ?)
    --CURRENT_DATE - Interval '1 day'

-- OPSLAAN ALS KCM_IRIS_REPARATIEINTAKE

(363 row(s) affected)

Total execution time: 00:01:38.570

case_id,code,description,created_datetime,lastchanged_datetime,direction_id,direction,channel_id,channel,subject_id,subject_code,subject,status_id,casestatus,created_user_wrkn,created_user_fullname,index_object_id,huurder_oge,huurder_rlts,department
2543052,1,Reperatieverzoek geregistreerd zonder planning,2021-07-01T08:34:55.437000,2021-07-01T08:34:55.437000,1,Huurder,1,Telefoon,141,TECHNTECHN,DAGEONDERH Reparatieverzoek,3,Afgehandeld,WRKN-01648,Robin van Vulpen,2544067,OGEH-0037935,RLTS-0036893,Intake
2543053,1,Reparatieverzoek met planning geregistreerd,2021-07-01T08:35:27.123000,2021-07-01T08:35:27.123000,1,Huurder,1,Telefoon,141,TECHNTECHN,DAGEONDERH Reparatieverzoek,3,Afgehandeld,WRKN-01233,Hennie van der Linden,2544068,OGEH-0039393,RLTS-0094623,Intake
2543056,1,Reparatieverzoek met planning geregistreerd,2021-07-01T08:37:50.603000,2021-07-01T08:37:50.603000,1,Huurder,1,Telefoon,141,TECHNTECHN,DAGEONDERH Reparatieverzoek,3,Afgehandeld,WRKN-01473,Selda Incedal,2544071,OGEH-0037493,RLTS-0024631,Intake
2543063,1,Reparatieverzoek met planning geregistreerd,2021-07-01T08:42:26.057000,2021-07-01T08:42:26.057000,1,Huurder,1,Telefoon,141,TECHNTECHN,DAGEONDERH Reparatieverzoek,3,Afgehandeld,WRKN-01473,Selda Incedal,2544078,OGEH-0035359,RLTS-0023336,Intake
2543065,1,Reparatieverzoek met planning geregistreerd,2021-07-01T08:44:07.080000,2021-07-01T08:44:07.080000,1,Huurder,1,Telefoon,141,TECHNTECHN,DAGEONDERH Reparatieverzoek,3,Afgehandeld,WRKN-01595,Nohad Cherif,2544080,OGEH-0018093,RLTS-0012282,Intake
2543068,1,Reparatieverzoek met planning geregistreerd,2021-07-01T08:46:57.947000,2021-07-01T08:46:57.947000,1,Huurder,1,Telefoon,141,TECHNTECHN,DAGEONDERH Reparatieverzoek,3,Afgehandeld,WRKN-01595,Nohad Cherif,2544083,OGEH-0053450,RLTS-0138209,Intake
2543069,1,Reparatieverzoek met planning geregistreerd,2021-07-01T08:47:18.813000,2021-07-01T08:47:18.813000,1,Huurder,1,Telefoon,141,TECHNTECHN,DAGEONDERH Reparatieverzoek,3,Afgehandeld,WRKN-01473,Selda Incedal,2544084,OGEH-0032158,RLTS-0028421,Intake
2543070,1,Reparatieverzoek met planning geregistreerd,2021-07-01T08:48:11.013000,2021-07-01T08:48:11.013000,1,Huurder,1,Telefoon,141,TECHNTECHN,DAGEONDERH Reparatieverzoek,3,Afgehandeld,WRKN-01233,Hennie van der Linden,2544085,OGEH-0006016,RLTS-0201304,Intake
2543072,1,Reparatieverzoek met planning geregistreerd,2021-07-01T08:48:46.127000,2021-07-01T08:48:46.127000,1,Huurder,1,Telefoon,141,TECHNTECHN,DAGEONDERH Reparatieverzoek,3,Afgehandeld,WRKN-01385,Emine Acikbas - Incedal,2544087,OGEH-0039454,RLTS-0034016,Intake
2543081,1,Reperatieverzoek geregistreerd zonder planning,2021-07-01T08:54:21.447000,2021-07-01T08:54:21.447000,1,Huurder,1,Telefoon,141,TECHNTECHN,DAGEONDERH Reparatieverzoek,3,Afgehandeld,WRKN-01725,Rachita Raghoebarsing,2544096,OGEH-0004908,RLTS-0233590,Intake


## Oplaan excl output als "\\\\s-dwh2012-DB\\<span style="font-size:13px;background-color:rgb(214, 235, 255);">e$\STAEDION\INLEESBESTANDEN\KCM_IRIS_REPARATIE_INTAKE.xlsx</span>

## Tussentabel vullen met data uit Iris <mark>\=\> S-DWH2012-B</mark>

schakel over naar s-dwh2012-db

In [4]:
use empire_staedion_data 
go
Drop Table If exists kcm.STN421_Iris_Cases
; 

select	 [case_id]
      ,[code]
      ,[description]
      ,[created_datetime]
      ,[lastchanged_datetime]
      ,[direction_id]
      ,[direction]
      ,[channel_id]
      ,[channel]
      ,[subject_id]
      ,[subject_code]
      ,[subject]
      ,[status_id]
      ,[casestatus]
      ,[created_user_wrkn]
      ,[created_user_fullname]
      ,[index_object_id]
      ,[huurder_oge]
      ,[huurder_rlts]
into	empire_staedion_data.kcm.STN421_Iris_Cases
from	[KCM_IRIS_REPARATIE_INTAKE]...[Sheet1$]
--from	[FF]...[Sheet1$]
GO



Commands completed successfully.

(363 rows affected)

Total execution time: 00:00:01.226